<a href="https://colab.research.google.com/github/rposhala/Tic-Tac-Toe-game-using-Reinforcement-learning/blob/master/TicTacToe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt

In [0]:
def print_game(s):
    
    print('     --------------')
    print('       {} | {} | {}'.format(s[0], s[1], s[2]))
    print('     --------------')
    print('       {} | {} | {}'.format(s[3], s[4], s[5]))
    print('     --------------')
    print('       {} | {} | {}'.format(s[6], s[7], s[8]))
    print('     --------------')
    

In [0]:
def check_winner(game):
    # print("game_winner")
    winner = ''
    checkfor = [[0, 1, 2], [3, 4, 5], [6, 7, 8], [0, 3, 6], [1, 4, 7], [2, 5, 8], [0, 4, 8], [2, 4, 6]]
    for line in checkfor:
        s = str(game[line[0]]) + str(game[line[1]]) + str(game[line[2]])
        if s == 'XXX':
            # winner = 'X'
            
            winner = 'The winner is PLAYER 1'
            break
        elif s == 'OOO':
            # winner = 'O'
            winner = 'The winner is PLAYER 2'
            break
    # print(game)
    # print([type(a) for a in game])
    if not any(a for a in game if type(a) != str) and 0 not in game and winner == '':
        winner = 'No winner, its a tie'
    
    
    return winner

In [0]:
def inputidx(a):
  try :
    idx = int(input('Choose move number: '))
    if idx in a:
      return int(idx)
    else :
      # print(a)
      print("please enter a number from the unoccupied places")
      idx = inputidx(a)
  except:
    # print(a)
    print("please enter a number")
    idx = inputidx(a)
  return idx

In [0]:

class Sarsa_Agent:
    def __init__(self, name, exploration=0.3):
        self.name = name
        self.terminate = None
        self.exploration_rate = exploration
        self.discounted_gamma = 0.9
        self.learning_rate = 0.2
        self.states = []  # record all positions taken
        self.dict_state_value = {}  # state -> value
        
    def NextStep(self, available_positions, current_board_state, turn):
        if np.random.uniform(0, 1) <= self.exploration_rate:
            # take random action
            idx = np.random.choice(len(available_positions))
            action = available_positions[idx]
        else:
            value_max = -999
            for i in available_positions:
                next_board_state = current_board_state.copy()
                next_board_state[i] = turn
                next_boardHash = self.getHash(next_board_state)
                value = 0 if self.dict_state_value.get(next_boardHash) is None else self.dict_state_value.get(next_boardHash)
                if value >= value_max:
                    value_max = value
                    action = i
        # print("{} takes action {}".format(self.name, action))
        return action


    def getHash(self, game):
        gameHash = str(game)
        return gameHash


    # append a hash state
    def addState(self, state):
        self.states.append(state)

    # at the end of game, backpropagate and update states value
    def feedReward(self, reward):
        for st in reversed(self.states):
            if self.dict_state_value.get(st) is None:
                self.dict_state_value[st] = 0
            self.dict_state_value[st] += self.learning_rate * (self.discounted_gamma * reward - self.dict_state_value[st])
            reward = self.dict_state_value[st]

    def reset(self):
        self.states = []

    def savePolicy(self):
        if os.path.exists('policy_' + str(self.name)):
          obj = open('policy_' + str(self.name), 'rb')
          existing_policies = pickle.load(obj)
          for i in existing_policies.keys():
            self.dict_state_value[i] = existing_policies[i]
            
        write_file = open('policy_' + str(self.name), 'wb')
        pickle.dump(self.dict_state_value, write_file)
        write_file.close()

    def loadPolicy(self, file):
        read_file = open(file, 'rb')
        self.dict_state_value = pickle.load(read_file)
        read_file.close()

In [0]:
def getHash(game):
  return str(game)

In [0]:
def reward(player1,player2,winner):
  # print(winner)
  if winner == 'The winner is PLAYER 1':
    player1.feedReward(1)
    player2.feedReward(0)
  elif winner == 'The winner is PLAYER 2':
    player1.feedReward(0)
    player2.feedReward(1)
  else :
    player1.feedReward(0.1)
    player2.feedReward(0.5)

In [0]:
def agent_training(player1,player2,rounds):
	p1 = 0
	p2 = 0
	tie = 0
	p1_plot = 0
	p2_plot = 0
	tie_plot = 0
	plot_p1 = []
	plot_p2 = []
	iterations_count =[]
	plot_tie = []
	for i in range(rounds):
		w = ''
		s = list(range(9))
		sav = np.zeros(9)
		avail = s.copy()
		a = s.copy()
		turn = ''
		while(w == ''):
			# Player 1 turn to play
			turn = 'X'
			p1_action = player1.NextStep(a, sav, 1)
			sav[p1_action] = 1
			s[p1_action] = turn
			a.remove(p1_action)
			board_hash = getHash(sav)
			player1.addState(board_hash)
			win = check_winner(s)
			if win != '':
				if win == 'No winner, its a tie':
					tie+=1
					tie_plot += 1
				else :
					p1+=1
					p1_plot+=1
				reward(player1,player2,win)
				player1.reset()
				player2.reset()
				break
			else:
	      	# Player 2 turn to play
				turn = 'O'
				p2_action = player1.NextStep(a, sav, -1)
				sav[p2_action] = -1
				s[p2_action] = turn
				a.remove(p2_action)
				board_hash = getHash(sav)
				player2.addState(board_hash)
				win = check_winner(s)
				if win != '':
					if win == 'No winner, its a tie':
						tie+=1
						tie_plot += 1
					else :
						p2+=1
						p2_plot+=1
					reward(player1,player2,win)
					player1.reset()
					player2.reset()
					break
		if (i+1)%500 == 0:
			iterations_count.append(i+1)
			plot_p1.append(p1_plot)
			plot_p2.append(p2_plot)
			plot_tie.append(tie_plot)
			p1_plot = 0
			p2_plot = 0
			tie_plot = 0

	fig = plt.figure()
	plot_figure = fig.add_subplot()
	plot_figure.plot(iterations_count, plot_p1, color='orange')
	plot_figure.plot(iterations_count, plot_p2, color='blue')
	plot_figure.plot(iterations_count, plot_tie, color='green')
	fig.show()
	print("Total iterations: ",rounds)
	print("Player1 (X) wins: ",p1)
	print("Player2 (O) wins: ",p2)
	print("Tie: ",tie)

In [0]:
def humanvsagent(agent,name,whichplayer):
  w = ''
  s = list(range(9))
  a = s.copy()
  sav = np.zeros(9)
  print_game(s)
  turn = ''
  if whichplayer == 1:
    human_turn = 'X'
    agent_turn = 'O'
    h_no = 1
    a_no = -1
  else :
    human_turn = 'O'
    agent_turn = 'X'
    h_no = -1
    a_no = 1
  c = whichplayer
  while w == '':
    c += 1
    if c%2 == 1:
      turn = agent_turn
      turn1 = a_no
      # agent_action = agent.NextStep(a,s,turn)
      agent_action = agent.NextStep(a,sav,turn1)
      # print(agent_action)
      index = agent_action
      # s[agent_action] = turn
      # a.remove(agent_action)
    else :
      turn = human_turn
      turn1 = h_no
      human_action = inputidx(a)
      index = human_action

    sav[index] = turn1
    s[index] = turn
    a.remove(index)
    print_game(s)
    w = check_winner(s)
  return w


In [0]:
hplayer1 = Sarsa_Agent("hard_p1")
hplayer2 = Sarsa_Agent("hard_p2")
print("training...")
agent_training(hplayer1, hplayer2,50000)
print("done")
hplayer1.savePolicy()
hplayer2.savePolicy()

In [0]:
mplayer1 = Sarsa_Agent("medium_p1")
mplayer2 = Sarsa_Agent("medium_p2")
print("training...")
agent_training(mplayer1, mplayer2,7000)
print("done")
mplayer1.savePolicy()
mplayer2.savePolicy()

In [0]:
eplayer1 = Sarsa_Agent("easy_p1")
eplayer2 = Sarsa_Agent("easy_p2")
print("training...")
agent_training(eplayer1, eplayer2,1000)
print("done")
eplayer1.savePolicy()
eplayer2.savePolicy()

In [13]:
p1 = Sarsa_Agent("agent", exploration= 0)
p1.loadPolicy("policy_hard_p1") # policy_mplayer1
name = 'Rohith'
whichplayer = 2
result = humanvsagent(p1,name,whichplayer)
if result == "The winner is PLAYER 1":
  if whichplayer == 2:
    print("Agent wins!")
  else :
    print("Human wins!")
elif result == "The winner is PLAYER 2":
  if whichplayer == 1:
    print("Agent wins!")
  else :
    print("Human wins!")
else :
  print(result)

     --------------
       0 | 1 | 2
     --------------
       3 | 4 | 5
     --------------
       6 | 7 | 8
     --------------
     --------------
       0 | 1 | 2
     --------------
       3 | 4 | 5
     --------------
       X | 7 | 8
     --------------
Choose move number: 4
     --------------
       0 | 1 | 2
     --------------
       3 | O | 5
     --------------
       X | 7 | 8
     --------------
     --------------
       0 | 1 | 2
     --------------
       3 | O | 5
     --------------
       X | X | 8
     --------------
Choose move number: 1
     --------------
       0 | O | 2
     --------------
       3 | O | 5
     --------------
       X | X | 8
     --------------
     --------------
       0 | O | 2
     --------------
       3 | O | 5
     --------------
       X | X | X
     --------------
Agent wins!


In [0]:
p1 = Sarsa_Agent("agent", exploration= 0)
p1.loadPolicy("policy_medium_p1") # policy_mplayer1
name = 'Rohith'
whichplayer = 2
result = humanvsagent(p1,name,whichplayer)
if result == "The winner is PLAYER 1":
  if whichplayer == 2:
    print("Agent wins!")
  else :
    print("Human wins!")
elif result == "The winner is PLAYER 2":
  if whichplayer == 1:
    print("Agent wins!")
  else :
    print("Human wins!")
else :
  print(result)

In [0]:
p1 = Sarsa_Agent("agent", exploration= 0)
p1.loadPolicy("policy_easy_p1") # policy_mplayer1
name = 'Rohith'
whichplayer = 2
result = humanvsagent(p1,name,whichplayer)
if result == "The winner is PLAYER 1":
  if whichplayer == 2:
    print("Agent wins!")
  else :
    print("Human wins!")
elif result == "The winner is PLAYER 2":
  if whichplayer == 1:
    print("Agent wins!")
  else :
    print("Human wins!")
else :
  print(result)

In [0]:
p1 = Sarsa_Agent("agent", exploration= 0)
p1.loadPolicy("policy_hard_p2") # policy_mplayer1
name = 'Rohith'
whichplayer = 1
result = humanvsagent(p1,name,whichplayer)
if result == "The winner is PLAYER 1":
  if whichplayer == 2:
    print("Agent wins!")
  else :
    print("Human wins!")
elif result == "The winner is PLAYER 2":
  if whichplayer == 1:
    print("Agent wins!")
  else :
    print("Human wins!")
else :
  print(result)

In [0]:
p1 = Sarsa_Agent("agent", exploration= 0)
p1.loadPolicy("policy_medium_p2") # policy_mplayer1
name = 'Rohith'
whichplayer = 1
result = humanvsagent(p1,name,whichplayer)
if result == "The winner is PLAYER 1":
  if whichplayer == 2:
    print("Agent wins!")
  else :
    print("Human wins!")
elif result == "The winner is PLAYER 2":
  if whichplayer == 1:
    print("Agent wins!")
  else :
    print("Human wins!")
else :
  print(result)

In [0]:
p1 = Sarsa_Agent("agent", exploration= 0)
p1.loadPolicy("policy_easy_p2") # policy_mplayer1
name = 'Rohith'
whichplayer = 1
result = humanvsagent(p1,name,whichplayer)
if result == "The winner is PLAYER 1":
  if whichplayer == 2:
    print("Agent wins!")
  else :
    print("Human wins!")
elif result == "The winner is PLAYER 2":
  if whichplayer == 1:
    print("Agent wins!")
  else :
    print("Human wins!")
else :
  print(result)

In [0]:
# def agent_training(player1,player2,rounds):
#   for i in range(rounds):
#     if i % 10000 == 0:
#       print("Rounds {}".format(i))
#     w = ''
#     s = list(range(9))
#     sav = np.zeros(9)
#     avail = s.copy()
#     a = s.copy()
#     # print_game(s)
#     turn = ''
#     while(w == ''):
#        # Player 1
#       turn = 'X'
#       # p1_action = player1.NextStep(a, s, turn)
#       ##
#       p1_action = player1.NextStep(a, sav, 1)
#       sav[p1_action] = 1

#       ##
#       s[p1_action] = turn
#       a.remove(p1_action)
#       # self.updateState(p1_action)
#       board_hash = getHash(sav)
#       player1.addState(board_hash)
#       # check board status if it is end
#       # print('p1',a)
#       win = check_winner(s)
#       if win != '':
#         # print('p1_p',a)
#         reward(player1,player2,win)
#         player1.reset()
#         player2.reset()
#         # self.reset()
#         break
#       else:
        
#         turn = 'O'
#         # p2_action = player1.NextStep(a, s, turn)
#         # take action and upate board state
#         ##
#         p2_action = player1.NextStep(a, sav, -1)
#         sav[p2_action] = -1
#         ##
#         s[p2_action] = turn
#         a.remove(p2_action)
#         # self.updateState(p1_action)
#         board_hash = getHash(sav)
#         player2.addState(board_hash)
#         # print('player2',a)
#         win = check_winner(s)
#         if win != '':
#           # print('p2_p',a)
#           reward(player1,player2,win)
#           player1.reset()
#           player2.reset()
#           # self.reset()
#           break
